In [ ]:
#base_packs
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn.preprocessing as skp
import sklearn.decomposition as skd
import statsmodels.api as sm
import sklearn.manifold as skm
import sklearn.cluster as skc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from statsmodels.stats.diagnostic import het_white
import ruptures as rpt
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report
import statsmodels.api as sm
import sklearn.preprocessing as skp
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
import statsmodels.discrete.discrete_model as sm
from sklearn.cluster import KMeans
import gc
import warnings
from scipy.optimize import OptimizeWarning
import time

In [ ]:
dt = 'V:/Departamentos/DVM/Transformação Digital/Projetos/064 - Otimização de Processo com IA no VOD (Ciência de Dados)/VOD_JUL_2024.xlsx'
dt = pd.read_excel(dt)
dt = pd.DataFrame(dt)
dt['Tempo de depressão de vácuo(min)'].replace(',' , '.', inplace = True)
dt['Tempo de depressão de vácuo(min)']= pd.to_numeric(dt['Tempo de depressão de vácuo(min)'] , errors = 'coerce')

dt['Teor de C após a descarburação(%)'].replace(',' , '.', inplace = True)
dt['Teor de C após a descarburação(%)']= pd.to_numeric(dt['Teor de C após a descarburação(%)'] , errors = 'coerce')
dt.drop(columns=['DATA', 'Agitação média do TruStir durante o sopro de O2','Agitação média do TruStir durante a descarburação'], inplace=True)
dt.replace('Boa', 1 , inplace = True)
dt.replace('Ruim', 0 , inplace = True)
dt = pd.get_dummies(dt)
for i in dt.columns:
    median_value = dt[i].median()  
    dt[i].fillna(median_value, inplace=True)
dt = dt.replace({False: 0, True: 1})


In [ ]:
##normalization
scaler = skp.RobustScaler()
dt_k = scaler.fit_transform(dt)
kmeans = KMeans(n_clusters= 400, random_state=0)

# Fit the model to the data
kmeans.fit(dt_k)

# Get cluster centers and labels
centers = kmeans.cluster_centers_
labels = kmeans.labels_
dt['cluster'] = labels 
tag =pd.DataFrame(dt['cluster'])
tag = pd.get_dummies(dt['cluster'], prefix='cluster')
tag.replace({False: 0, True: 1})
## opt
dt['opt'] = 0
dt.loc[
    (dt['Teor de C após a descarburação(%)'] < 0.008) &
(dt['Temperatura do aço após a descarburação(°C)'] > 1680) 
    ,
    'opt'
] = 1


In [ ]:
#priorizar o teor de c após a descarburação na otimização.
# optimização:
dt['opt'] = 0
dt.loc[
    (dt[''] <= 0.008) &
(dt[''] >= 1680) &
(dt[''] <= 1760)
    ,
    'opt'
] = 1

In [ ]:
##
#Logit:
x_ = dt[[ ]]
x_x = tag[tag.columns]


y_ = dt['opt']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_x, y_, test_size=0.5)
# Decision tree classifier:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(x_x, y_)

result = permutation_importance(clf, x_x, y_, n_repeats=10, random_state=42)
importance = result.importances_mean
importance = pd.DataFrame(importance)
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = [dt.columns.to_numpy]
impor_var = importance[importance[0] > importance[0].mean()]
impor_var_new = impor_var.reset_index().rename(columns={'index': 'old_index'})

model= sm.Logit(y_,x_)
co =model.fit().params
coe = pd.DataFrame(co)
coe.columns = ['coeff']
exp_coeff =   np.exp(coe)/(np.exp(coe) + 1) 
exp_coeff = pd.DataFrame(exp_coeff)
exp_coeff.columns = ['exp_coeff']
exp_coeff
results = model.fit(method = 'bfgs',cov_type='HC3' )
results_summary = results.summary()
results_as_html = results_summary.tables[1].as_html()
df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
df_results['exp_coeff'] = exp_coeff
df_results
impact_var = df_results.loc[(df_results['P>|z|'] < 0.05 )]

In [ ]:
clus_tab = dt.groupby(['cluster']).median()
clust_test = clus_tab.loc[impor_var_new['old_index']]
clust_test_test = clust_test[(clust_test['opt'] == clust_test['opt'].max())] 

In [ ]:
clust_test_test[[]].mean()

In [ ]:
clust_test_test[[]].mean()

In [ ]:
5.215138 , 450.263303 ,  17.880523 ,    6.347615,  178.655963, 1699.509174
4.9000 , 0 , 17.8830 , 6.4500 , 187.0000,1684

In [ ]:
## primaira volta: # inclui a temperatura no final da corrida
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore')

columns_ = [
   ]
   
stp = [[5.215138 , 450.263303 ,  17.880523 ,    6.347615,  178.655963, 1699.509174]]
contr = pd.DataFrame(stp, columns = columns_)
# controle df:
ent = dt[['']]
ent_ = ent.sample(n=1)
contr.reset_index(drop=True, inplace=False)
tt2 = pd.concat([ent_,contr], axis=1)
tt2.fillna(0)
tt2 =pd.DataFrame(tt2.sum())
tt2 = tt2.T
tt2_test = tt2[[
                ]]
x_ = dt[[]]
y_ = dt['opt']
#tt2_test.columns = x_train_3.columns
x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_,y_, test_size = 0.9)
model = LogisticRegression(penalty='l2', solver='liblinear',class_weight= 'balanced')
model.fit(x_, y_)
y_pred2 = model.predict(tt2)
y_pred_prob2 = model.predict_proba(tt2)
y_pred_score2 = model.score(x_, y_)
###
## inferência causal:
from sklearn.linear_model import LogisticRegression
import statsmodels.discrete.discrete_model as sm
model= sm.Logit(y_train_3, x_train_3)
co =model.fit(method = 'bfgs',cov_type='HC3').params
coe = pd.DataFrame(co)
coe.columns = ['coeff']
exp_coeff =   np.exp(coe)/(np.exp(coe) + 1) 
exp_coeff = pd.DataFrame(exp_coeff)
exp_coeff.columns = ['exp_coeff']
exp_coeff
results = model.fit(method = 'bfgs',cov_type='HC3')
results_summary = results.summary()
results_as_html = results_summary.tables[1].as_html()
df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
df_results['exp_coeff'] = exp_coeff
impact_var2 = df_results.loc[(df_results['P>|z|'] < 0.05 )]
###
results_list = []

# Iterate over each index in impact_var2
for i in impact_var2.index:
    value = impact_var2.loc[i, 'exp_coeff']
    
    # Check if the column exists in contr
    if i in contr.columns:
        correction = tt2_test[i].mean()
    
        # Calculate the result based on the value
        if value < 0.5:
            result = (value - 0.5) * correction
        elif value > 0.5:
            result = (value - 0.5) * correction
        else:
            result = 0  # Handle the case where value == 0.5

        # Append result to the list
        results_list.append({'Column': i, 'Alteração': result})

# Create a DataFrame from the results list
results_df_ = pd.DataFrame(results_list)

# Filter results_df_ to include only columns present in contr
results_df_

####
dd = results_df_.T
new_header = dd.iloc[0]  # The first row

# Step 2: Create a new DataFrame with the new headers
dd = dd[1:]  # Remove the first row from the DataFrame
dd.columns = new_header  # Set the new header as column names

# Reset index if needed
dd.reset_index(drop=True, inplace=True)
##
common_columns = list(set(contr.columns) & set(dd.columns))
contr_common = contr[common_columns]
dd_common = dd[common_columns]

# Step 3: Concatenate the DataFrames and sum the values
# Concatenate DataFrames
combined_df = pd.concat([contr_common, dd_common], ignore_index=True)

# Sum the values for the common columns
final = combined_df.sum().reset_index()
tt = final.T
new_header = tt.iloc[0]  # The first row

# Step 2: Create a new DataFrame with the new headers
tt = tt[1:]  # Remove the first row from the DataFrame
tt.columns = new_header  # Set the new header as column names

# Reset index if needed
tt.reset_index(drop=True, inplace=True)
###
contr.reset_index(drop = True)
tt.reset_index(drop = True)

# Step 2: Identify common columns
common_columns = contr.columns.intersection(tt.columns)

# Step 3: Update values in df_main for the common columns
contr.update(tt[common_columns])

# Reset index if needed
contr.reset_index(inplace=True)
###
contr.drop(columns=['index'], inplace = True)

In [ ]:
y_pred_prob2

In [ ]:
ent_.index

In [ ]:
stp

In [ ]:
# Initialize the loop variables
warnings.filterwarnings('ignore')
condition_met = False
start_time = time.time()
time_limit = 300 

iteration = 0

while not condition_met:
    try:
        # Track elapsed time
        elapsed_time = time.time() - start_time
        if elapsed_time > time_limit:
            print("Time limit exceeded. Stopping the loop.")
            break
        
        iteration += 1
        print(f"Iteration: {iteration}, Time elapsed: {elapsed_time:.2f} seconds")

        # Reset index for contr DataFrame
        contr.reset_index(drop=True, inplace=True)

        # Concatenate and process DataFrames
        tt2 = pd.concat([ent_, contr], axis=1)
        tt2.fillna(0, inplace=True)
        tt2 = pd.DataFrame(tt2.sum()).T

        tt2_test = tt2[[
                ]]

        x_ = dt[[]]
        y_ = dt['opt']

        x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_, y_, test_size=0.9)
        model = LogisticRegression(penalty='l2', solver='liblinear', class_weight='balanced')
        model.fit(x_train_3, y_train_3)
        y_pred2 = model.predict(tt2)
        y_pred_prob2 = model.predict_proba(tt2)[:, 1]  # Get the probability of the positive class
        y_pred_score2 = model.score(x_test_3, y_test_3)

        # Inferência causal
        model = sm.Logit(y_train_3, x_train_3)
        co = model.fit(method='bfgs', cov_type='HC3').params
        coe = pd.DataFrame(co)
        coe.columns = ['coeff']
        exp_coeff = np.exp(coe) / (np.exp(coe) + 1)
        exp_coeff = pd.DataFrame(exp_coeff)
        exp_coeff.columns = ['exp_coeff']
        results = model.fit(method='bfgs', cov_type='HC3')
        results_summary = results.summary()
        results_as_html = results_summary.tables[1].as_html()
        df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
        df_results['exp_coeff'] = exp_coeff
        impact_var2 = df_results.loc[df_results['P>|z|'] < 0.05]

        results_list = []
        for i in impact_var2.index:
            value = impact_var2.loc[i, 'exp_coeff']

            if i in contr.columns:
                correction = tt2_test[i].mean()

                if value < 0.5:
                    result = (value - 0.5) * correction
                elif value > 0.5:
                    result = (value - 0.5) * correction
                else:
                    result = 0

                results_list.append({'Column': i, 'Alteração': result})

        results_df_ = pd.DataFrame(results_list)
        dd = results_df_.T
        new_header = dd.iloc[0]
        dd = dd[1:]
        dd.columns = new_header
        dd.reset_index(drop=True, inplace=True)

        common_columns = list(set(contr.columns) & set(dd.columns))
        contr_common = contr[common_columns]
        dd_common = dd[common_columns]

        combined_df = pd.concat([contr_common, dd_common], ignore_index=True)
        final = combined_df.sum().reset_index()
        tt = final.T
        new_header = tt.iloc[0]
        tt = tt[1:]
        tt.columns = new_header
        tt.reset_index(drop=True, inplace=True)

        contr.reset_index(drop=True, inplace=True)
        tt.reset_index(drop=True, inplace=True)

        common_columns = contr.columns.intersection(tt.columns)
        contr.update(tt[common_columns])
        contr.reset_index(inplace=True)
        contr.drop(columns=['index'], inplace=True)

        gc.collect()

        # Check if the condition is met
        if np.all(y_pred_prob2 >= 0.99) and np.all(y_pred2 == 1):
            condition_met = True

    except Exception as e:
        print(f"An error occurred: {e}")
        # Handle or log the error as needed
        continue

# After the loop ends, display the results
print("Final Results:")

print("Model Performance:")
print(f"Accuracy Score: {y_pred_score2:.4f}")

print("\nPredicted Classes:")
print(pd.DataFrame({'Predicted Class': y_pred2}))

print("\nPredicted Probabilities:")
print(pd.DataFrame({'Predicted Probability': y_pred_prob2}))

print("\nResults DataFrame:")
print(results_df_)

print("\nUpdated 'contr' DataFrame:")
print(contr)

In [ ]:
y_pred_prob2

In [ ]:
y_pred2

In [ ]:
ent_.index

In [ ]:
tt2[columns_]

In [ ]:
stp